In [29]:
import csv
import glob
import os
from collections import namedtuple
CandidateInfoTuple = namedtuple(
    'CandidateInfoTuple',
    'isNodule_bool, diameter_mm, series_uid, center_xyz',
)
path = 'D:/LUNA16/'
mhd_list = glob.glob(path + '/subset*/*.mhd')
    # 截取文件名称，不带后缀名
presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}
diameter_dict = {}
with open(path+'CSVFILES/annotations.csv', "r") as f:
    for row in list(csv.reader(f))[1:]:
        series_uid = row[0]
        annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
        annotationDiameter_mm = float(row[4])
        diameter_dict.setdefault(series_uid, []).append(
            (annotationCenter_xyz, annotationDiameter_mm)
        )


In [30]:
candidateInfo_list = []
with open(path +'CSVFILES/candidates.csv', "r") as f:
    for row in list(csv.reader(f))[1:]:
           series_uid = row[0]
           if series_uid not in presentOnDisk_set :
               continue
           isNodule_bool = bool(int(row[4]))
           candidateCenter_xyz = tuple([float(x) for x in row[1:4]])
           candidateDiameter_mm = 0.0
           for annotation_tup in diameter_dict.get(series_uid, []):
               annotationCenter_xyz, annotationDiameter_mm = annotation_tup
               for i in range(3):
                   delta_mm = abs(candidateCenter_xyz[i] - annotationCenter_xyz[i])
                   if delta_mm > annotationDiameter_mm / 4:
                       break
                   else:
                       candidateDiameter_mm = annotationDiameter_mm
                       break
    candidateInfo_list.append(CandidateInfoTuple(
           isNodule_bool,
           candidateDiameter_mm,
           series_uid,
           candidateCenter_xyz,
        ))